# Generate interfaces using the zsl generator



In [1]:
import sys
sys.path.append('./zsl_generator')

from zsl_match import create_zsl_match
from zsl_analyzer import compute_zsl_matches
from interface_from_zsl import interface_from_zsl

from pymatgen.core.structure import Structure
from pymatgen.core.interface import fix_pbc

from pymatgen.ext.matproj import MPRester

### Retrieve structure from materials project

In [ ]:
# Replace 'YOUR_API_KEY' with your actual Materials Project API key
api_key = 'YOUR_API_KEY'

# Initialize MPRester with your API key
with MPRester(api_key) as mpr:

    # Fetch SiO2 (mp-7000) and Si (mp-149) structures by their Materials Project IDs
    siO2_structure_a = mpr.get_structure_by_material_id("mp-7000")  # alpha-quartz
    siO2_structure_b = mpr.get_structure_by_material_id("mp-6922")  # beta-quartz
    si_structure = mpr.get_structure_by_material_id("mp-149")     # Si

    # Print the structures
    print("SiO2 Structure Alpha:")
    print(siO2_structure_a)
    print("SiO2 Structure Alpha:")
    print(siO2_structure_b)

    print("\nSi Structure:")
    print(si_structure)

    # # Optionally, save them as CIF files for later use
    # siO2_structure.to(fmt="cif", filename="SiO2_structure.cif")
    # si_structure.to(fmt="cif", filename="Si_structure.cif")

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

Retrieving MaterialsDoc documents:   0%|          | 0/1 [00:00<?, ?it/s]

SiO2 Structure Alpha:
Full Formula (Si3 O6)
Reduced Formula: SiO2
abc   :   4.914966   4.914966   5.431301
angles:  90.000000  90.000000 119.999991
pbc   :       True       True       True
Sites (9)
  #  SP            a          b         c    magmom
---  ----  ---------  ---------  --------  --------
  0  Si     0.531089   0.531089  0               -0
  1  Si    -0          0.468911  0.666667        -0
  2  Si     0.468911  -0         0.333333        -0
  3  O      0.269223   0.413394  0.784891         0
  4  O      0.586606   0.855828  0.118225         0
  5  O      0.144172   0.730777  0.451558         0
  6  O      0.413394   0.269223  0.215109         0
  7  O      0.730777   0.144172  0.548442         0
  8  O      0.855828   0.586606  0.881775         0
SiO2 Structure Alpha:
Full Formula (Si3 O6)
Reduced Formula: SiO2
abc   :   5.061089   5.061088   5.540501
angles:  90.000000  90.000000 119.999997
pbc   :       True       True       True
Sites (9)
  #  SP           a         b 

### Compute Matches

The `compute_zsl_matches` function uses pymatgen’s ZSL algorithm to generate lattice-matched interfaces between a substrate and a film. For each match, it computes strain metrics, transformation matrices, and saves the results (both as a summary CSV and individual match files) for further use in interface construction and analysis.

In [ ]:
substrate_miller = (1, 1, 0)
film_miller = (1, 0, 0)

# Format Miller indices into strings
substrate_str = ''.join(map(str, substrate_miller))
film_str = ''.join(map(str, film_miller))
calc_name = f"Si_{substrate_str}_SiO2_{film_str}_alpha"

compute_zsl_matches(
    substrate=si_structure,
    substrate_miller=substrate_miller,
    film=siO2_structure_a,
    film_miller=film_miller,
    calc_name=calc_name
)


### Create Match

The `create_zsl_match` function generates a specific lattice match between a crystalline substrate and film using pymatgen's ZSL algorithm. Given a `match_id`, it constructs the corresponding match object, which contains supercell vectors, transformation matrices, and strain data. This match can later be used to generate an actual atomic interface. The function also prints the number of available terminations (i.e., atomic layers) for each material at the interface.

In [34]:
match_id = 0
zsl_match = create_zsl_match(substrate=si_structure, substrate_miller=substrate_miller, film=siO2_structure_a, film_miller=film_miller, match_id=match_id) 

[('Si_Pmmm_1', 'Si_P4/mmm_1'), ('O2_Pmmm_1', 'Si_P4/mmm_1')]


### Generate Interface

The `interface_from_zsl` function builds a 3D atomic interface structure between a substrate and a film using a selected lattice match. It allows precise control over interface terminations, film/substrate thicknesses, vertical alignment (z-shift), and vacuum. It trims excess atomic layers, applies optional mirroring of the substrate, and returns a complete `pymatgen.Interface` object for further simulation or analysis.

In [ ]:
termination=('O2_Pmmm_1', 'Si_P4/mmm_1')

gap = 2
vacuum_over_film = 2
film_thickness = 6
film_shift = 0.0
substrate_thickness = 8
substrate_shift = 0.0
in_plane_offset = (0.0, 0.0)
substrate_flip = False

interface = interface_from_zsl(substrate=si_structure, substrate_miller=substrate_miller, film=siO2_structure_a, film_miller=film_miller,zsl_match=zsl_match,
                   termination=termination,gap=gap, vacuum_over_film=vacuum_over_film,film_thickness=film_thickness,
                   film_shift=film_shift,substrate_thickness=substrate_thickness, substrate_shift=substrate_shift,
                   in_plane_offset=in_plane_offset,substrate_flip=substrate_flip)

### Save Structure

In [ ]:
import re
from pymatgen.io.ase import AseAtomsAdaptor
from ase.io import write

# Sanitize for filenames
def sanitize_string(s):
    return re.sub(r'[^\w\-]', '_', s)

# Updated filename generator with supercell info
def generate_unique_filename(
    substrate_name, film_name,
    substrate_miller, film_miller,
    match_id, termination, gap, 
    vacuum_over_film, film_thickness, substrate_thickness, 
    substrate_shift, in_plane_offset, substrate_flip,
    supercell=(1, 1, 1)
):
    substrate_miller_str = f"{substrate_name}_{substrate_miller[0]}{substrate_miller[1]}{substrate_miller[2]}"
    film_miller_str = f"{film_name}_{film_miller[0]}{film_miller[1]}{film_miller[2]}"
    match_str = f"{match_id}"

    term_0 = sanitize_string(termination[0])
    term_1 = sanitize_string(termination[1])
    termination_str = f"term_{term_0}_{term_1}"

    supercell_str = f"supercell_{supercell[0]}x{supercell[1]}x{supercell[2]}"
    params_str = (
        f"gap_{gap}_vac_{vacuum_over_film}_filmthick_{film_thickness}"
        f"_substratethick_{substrate_thickness}_substrate_shift_{substrate_shift}"
        f"_offset_{in_plane_offset[0]}_{in_plane_offset[1]}_flip_{'F' if not substrate_flip else 'T'}"
    )

    unique_filename = f"{substrate_miller_str}_{film_miller_str}_{termination_str}_{match_str}_{supercell_str}_{params_str}.cif"
    return unique_filename

# === INPUTS ===
substrate_name = 'Si'
film_name = 'SiO2_alpha'
supercell = (1, 1, 1)  # Repeat in x-direction


# Generate filename
filename = generate_unique_filename(
    substrate_name=substrate_name,
    film_name=film_name,
    substrate_miller=substrate_miller,
    film_miller=film_miller,
    match_id=match_id,
    termination=termination,
    gap=gap,
    vacuum_over_film=vacuum_over_film,
    film_thickness=film_thickness,
    substrate_thickness=substrate_thickness,
    substrate_shift=substrate_shift,
    in_plane_offset=in_plane_offset,
    substrate_flip=substrate_flip,
    supercell=supercell  # New argument
)

print("Saving to:", filename)

# Convert to ASE Atoms and repeat
atoms = AseAtomsAdaptor.get_atoms(interface)
atoms = atoms.repeat(supercell)

# File paths
folder_name = '/path/to/folder/' 
file_path = folder_name + filename

# Save as CIF and LAMMPS
write(file_path, atoms, format="cif")
write(file_path.replace('.cif', '.data'), atoms, format="lammps-data")

# Optionally preview structure
# view(atoms)


Saving to: Si_110_SiO2_alpha_100_term_O2_Pmmm_1_Si_P4_mmm_1_0_supercell_1x1x1_gap_2_vac_2_filmthick_6_substratethick_8_substrate_shift_0.0_offset_0.0_0.0_flip_F.cif
